# VacationPy

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Grand Gaube,-20.0064,57.6608,78.01,76,0,5.01,MU,1612492432
1,Atuona,-9.8000,-139.0333,78.78,72,1,11.25,PF,1612492420
2,Longyearbyen,78.2186,15.6401,15.80,73,75,12.66,SJ,1612492416
3,Kapaa,22.0752,-159.3190,69.80,56,90,5.75,US,1612492419
4,Katsuura,35.1333,140.3000,51.01,36,40,3.44,JP,1612492423
...,...,...,...,...,...,...,...,...,...
542,Zemio,5.0314,25.1361,68.68,75,31,2.24,CF,1612493099
543,Richards Bay,-28.7830,32.0377,73.58,89,89,8.10,ZA,1612492998
544,Chazuta,-6.5742,-76.1367,75.52,84,100,0.67,PE,1612493099
545,Tabuk,17.4189,121.4443,76.33,72,43,5.26,PH,1612493099


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# configure gmaps with API key
gmaps.configure(api_key=g_key)

# setting locations and weight
cities_df = cities_df.dropna()

locations = cities_df[["Lat", "Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)

# Create a heatmap layer
fig = gmaps.figure(zoom_level=1.5, center=(20,20))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 max_intensity=max(cities_df["Humidity"]))
    
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [5]:
# create my ideal weather conditions: max temperature < 85 degrees but > 70,
# humidity between 30-50%, and low change of cloudiness.

ideal_weather_df = cities_df.loc[(cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 85) & 
                                 (cities_df["Humidity"] >= 30) & (cities_df["Humidity"] <= 50) & 
                                 (cities_df["Cloudiness"] <= 20)]

# drop any rows will null values
ideal_weather_df = ideal_weather_df.dropna()    

# display dataframe
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,Busselton,-33.6500,115.3333,71.01,45,1,4.00,AU,1612492412
113,Uthai Thani,15.3833,99.5500,82.99,48,0,5.46,TH,1612492998
150,Chittarkonda,18.1167,82.0833,70.57,44,0,0.56,IN,1612493007
334,Kwinana,-32.2500,115.7667,73.00,35,0,18.41,AU,1612493050
385,Taunggyi,20.7833,97.0333,70.74,40,0,1.59,MM,1612493062
387,‘Ewa Beach,21.3156,-158.0072,75.20,50,20,17.27,US,1612493062
396,Makakilo City,21.3469,-158.0858,75.20,50,20,17.27,US,1612493042
425,Bo Phloi,14.3236,99.5161,84.99,35,0,6.71,TH,1612493071
444,Monywa,22.1167,95.1333,72.77,38,4,6.76,MM,1612493075
496,Babu,24.4167,111.5167,76.50,33,0,4.41,CN,1612493088


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels within 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# create new hotel dataframe
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]].copy()

# add a new column for Hotel Name and set empty parameters to store data
hotel_df["Hotel Name"] = ""

# set parameters to search for hotels within 5000 meters
parameters = {
    "type": "lodging",
    "radius": 5000,
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    # set parameters to loop through each city's coordinates
    lat = row["Lat"]
    lng = row["Lng"]
    parameters["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make API request
    response = requests.get(base_url, params=parameters).json()
    
    # extract search results
    results = response["results"]
    
    # store search results to Hotel Name column
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except:
        print("no hotel found")
    
hotel_df

,City,Country,Lat,Lng,Hotel Name
17,Busselton,AU,-33.6500,115.3333,Observatory Guest House
113,Uthai Thani,TH,15.3833,99.5500,ดนัย​ สารสุ​ว​รรณ์​
150,Chittarkonda,IN,18.1167,82.0833,Government girls hostel
334,Kwinana,AU,-32.2500,115.7667,Rockingham Apartments
385,Taunggyi,MM,20.7833,97.0333,True Treasure Hotel
387,‘Ewa Beach,US,21.3156,-158.0072,Base Lodging
396,Makakilo City,US,21.3469,-158.0858,Marriott's Ko Olina Beach Club
425,Bo Phloi,TH,14.3236,99.5161,แอท เมืองพลอย รีสอร์ท At MuangPloy Resort
444,Monywa,MM,22.1167,95.1333,Monywa Hotel
496,Babu,CN,24.4167,111.5167,IU Hotel Hezhou Bus Terminal


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name:</dt><dd>{Hotel Name}</dd>
<dt>City:</dt><dd>{City}</dd>
<dt>Country:</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
pin_locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Plot hotels on top of the humidity heatmap with pins containing Hotel Name, City, and Country.

# Create a marker_layer using the hotel list to fill the info box
fig = gmaps.figure(zoom_level=1.5, center=(20,20))
markers = gmaps.marker_layer(pin_locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure with hotel names on top of heat map
fig

Figure(layout=FigureLayout(height='420px'))